In [103]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, GlobalAveragePooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import BatchNormalization
from sklearn.metrics import classification_report


In [63]:
pip install opendatasets

Note: you may need to restart the kernel to use updated packages.


In [64]:
import opendatasets as od
od.download('https://www.kaggle.com/datasets/bilalakgz/brain-tumor-mri-dataset')

Skipping, found downloaded files in "./brain-tumor-mri-dataset" (use force=True to force download)


In [91]:
# Define constants
IMG_SIZE = (224, 224)
BATCH_SIZE = 32
DATASET_PATH = 'brain-tumor-mri-dataset' 
num_classes=4

In [92]:
datagen = ImageDataGenerator(
    rescale=1./4095,
    rotation_range=20,
    zoom_range=0.15,
    horizontal_flip=True,
    validation_split=0.2
)

train_data = datagen.flow_from_directory(
    DATASET_PATH,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='training'
)

val_data = datagen.flow_from_directory(
    DATASET_PATH,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='validation'
)

test_data = datagen.flow_from_directory(
    DATASET_PATH,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='validation',
    shuffle=False
)

Found 2729 images belonging to 1 classes.
Found 682 images belonging to 1 classes.
Found 682 images belonging to 1 classes.


In [97]:
model = Sequential([
    Conv2D(32, (3,3), activation='relu', input_shape=(224,224,3)),
    BatchNormalization(),  # Added
    MaxPooling2D(2,2),
    
    Conv2D(64, (3,3), activation='relu'),
    BatchNormalization(),  # Added
    MaxPooling2D(2,2),
    
    Conv2D(128, (3,3), activation='relu'),
    BatchNormalization(),  # Added
    MaxPooling2D(2,2),
    
    GlobalAveragePooling2D(),
    Dense(256, activation='relu'),
    Dropout(0.5),
    Dense(num_classes, activation='softmax')
])

In [98]:
model.compile(optimizer=Adam(learning_rate=1e-5),
              loss='categorical_crossentropy',
              metrics=['accuracy'])


In [99]:
early_stop = EarlyStopping(monitor='val_loss', patience=3)
checkpoint = ModelCheckpoint('model_70.h5', monitor='val_accuracy', save_best_only=True)

In [100]:
history = model.fit(
    train_data,
    validation_data=val_data,
    epochs=20,
    callbacks=[early_stop, checkpoint]
)


Epoch 1/20
86/86 [==============================] - 36s 394ms/step - loss: 5.9181 - accuracy: 0.4405 - val_loss: 5.6232 - val_accuracy: 0.0000e+00
Epoch 2/20
86/86 [==============================] - 34s 390ms/step - loss: 5.9488 - accuracy: 0.4558 - val_loss: 5.7720 - val_accuracy: 0.0000e+00
Epoch 3/20
86/86 [==============================] - 34s 391ms/step - loss: 6.0217 - accuracy: 0.4181 - val_loss: 5.8306 - val_accuracy: 0.0000e+00
Epoch 4/20
86/86 [==============================] - 34s 390ms/step - loss: 6.1350 - accuracy: 0.4075 - val_loss: 5.7929 - val_accuracy: 0.0000e+00


In [101]:
model.save('brain_tumor_model.h5')

In [105]:
# Test Accuracy and Metrics
test_loss, test_acc = model.evaluate(test_data)
print(f'\nTest Accuracy: {test_acc:.4f}')

# Predictions and Metrics
y_true = test_data.classes
y_pred = model.predict(test_data)
y_pred_classes = np.argmax(y_pred, axis=1)



22/22 [==============================] - 7s 303ms/step - loss: 5.7924 - accuracy: 0.0000e+00

Test Accuracy: 0.0000
22/22 [==============================] - 7s 301ms/step
